In [ ]:
# --- bootstrap so `import ts2eg` works before installation ---
import sys, os
for up in [os.getcwd(), os.path.dirname(os.getcwd()), os.path.dirname(os.path.dirname(os.getcwd()))]:
    src = os.path.join(up, "src")
    if os.path.isdir(src) and src not in sys.path:
        sys.path.insert(0, src)
        break
import ts2eg as gm
from ts2eg import extensions as ext

# CYBER_DEMO: Detector portfolio → EGT analysis (synthetic)

This notebook demonstrates the pipeline on **cybersecurity detector portfolios**.

**Players**: detectors/rule families (e.g., signature, anomaly, DNS, sandbox, EDR heuristics, ML).

**Signals `X`**: share of alert volume or compute/triage time per detector (columns sum to 1).

**Payoffs `v`**: **information-gain** toward a target `J` = negative *future incident cost* (dwell time / spread / escalations), estimated by rolling LOFO ablation.

We simulate an **adversary–defender** setting where attackers shift tactics; we fit a strategy-level operator `A` and look for **ESS** (stable detector mix) and **cycles** (skew in `A`).


In [ ]:
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from gameify_timeseries import info_gain_payoffs, estimate_A_from_series, find_ESS
print('Imports OK')

## 1) Simulate detectors and adaptive adversary
We model `N=6` detectors and a hidden `K=3`-archetype mix that the adversary responds to; defender routes triage capacity among detectors. The environment has day-of-week effects.

In [ ]:
def simulate_cyber(N=6, K=3, T=24*30, seed=0):  # hourly for ~1 month
    rng = np.random.default_rng(seed)
    # True detector archetypes (columns normalized)
    S = rng.random((N, K)); S /= (np.linalg.norm(S, axis=0, keepdims=True) + 1e-12)
    # Adversary–defender interaction among archetypes (skew → cycles)
    A_true = np.array([[0.0,  0.25, -0.18],
                       [-0.25, 0.0,  0.22],
                       [0.18, -0.22, 0.0]])
    x = np.full(K, 1.0/K)
    X = np.zeros((N, T))
    # Exogenous day-of-week baseline load
    t = np.arange(T)
    dow = (t // 24) % 7
    base = 0.5 + 0.2*(dow >= 1)*(dow <= 4)  # busier Tue–Thu
    noise = 0.10*rng.standard_normal((N, T))
    for tt in range(T):
        # Detector share vector with noise (rectified)
        X[:, tt] = np.maximum(S @ x + noise[:, tt], 1e-8)
        X[:, tt] /= X[:, tt].sum()
        # Hidden replicator-like update (attacker/defender cycle)
        u = A_true @ x; u -= u.mean()
        x = x + 0.10*(x * u) + 0.01*rng.random(K)
        x = np.maximum(x, 1e-8); x = x / x.sum()
    # Construct incident-cost proxy (lower is better); make J = -cost (higher better)
    w = np.array([0.6, 0.4, -0.2, -0.1, 0.3, -0.2])  # some detectors reduce cost; some add noise/FP
    incident_cost = (X.T @ (-w)) + 0.2*np.sin(2*np.pi*t/24) + 0.2*rng.standard_normal(T) + (dow==5)*0.4
    J = -incident_cost
    return S, X, J

S_true, X, J = simulate_cyber()
N, T = X.shape; K = 3
detectors = ['SIG','ANOM','DNS','SANDBOX','EDR','ML']
print('X shape:', X.shape, 'J length:', J.shape)

## 2) Payoffs = rolling LOFO information‑gain toward `J`
We compute `v = info_gain_payoffs(X, J, window=168)` (one-week window for hourly data).

In [ ]:
v = info_gain_payoffs(X, J, ridge=1e-2, window=168)
print('v shape:', v.shape)

## 3) Learn strategies via `nmf_on_X` and estimate `A`

In [ ]:
from gameify_timeseries import nmf_on_X
K = 3
S_hat, H = nmf_on_X(X, k=K, iters=300, seed=1, normalize='l2')
print('S_hat shape:', S_hat.shape)


## 4) Fit `A` and search for ESS

In [ ]:
est = estimate_A_from_series(S_hat, X, v, k=K, lambda_=1e-2)
A = est['A']
print('R^2:', round(est['R2'], 3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))
As, Aa = 0.5*(A + A.T), 0.5*(A - A.T)
print('||A_s||_F, ||A_a||_F =', float(np.linalg.norm(As)), float(np.linalg.norm(Aa)))

## 5) Null: circularly shift each detector's series
Break cross-detector coordination while preserving each row's autocorrelation.

In [ ]:
def circular_shift_rows(M):
    N, T = M.shape
    rng = np.random.default_rng(11)
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

X_null = circular_shift_rows(X)
S_null, _ = nmf_multiplicative(X_null, r=K, iters=300, seed=7)
S_null = S_null / (np.linalg.norm(S_null, axis=0, keepdims=True) + 1e-12)
v_null = info_gain_payoffs(X_null, J, ridge=1e-2, window=168)
est_null = estimate_A_from_series(S_null, X_null, v_null, k=K, lambda_=1e-2)
print('R^2 (null):', round(est_null['R2'], 3))
ess_null = [r for r in find_ESS(est_null['A'], tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (null):', len(ess_null))

## 6) Quick plots

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Inferred strategy mixture x(t)'); plt.legend(); plt.show()